## Importing Necessary Libraries

In [46]:
import cv2
import numpy as np
from pygame import mixer
from keras.models import load_model
import time
import psutil

## Initializing the Model, Eye and Face Cascades, Alarm Audio, Camera, and Text Font

In [47]:
# Loading the Model
model = load_model('model.h5')

# Initializing Left Eye, Right Eye, and Face Cascades from OpenCV
face_classifier = cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_alt.xml')
left_eye_classifier = cv2.CascadeClassifier('haarcascade/haarcascade_lefteye_2splits.xml')
right_eye_classifier = cv2.CascadeClassifier('haarcascade/haarcascade_righteye_2splits.xml')

# Initializing Pygame Mixer for Audio
mixer.init()
soundfile = 'alarm.wav'
sound = mixer.Sound(soundfile)

# Initializing the Camera
cap = cv2.VideoCapture(0)

# Font for the text
font = cv2.FONT_HERSHEY_SIMPLEX

## Initializing Necessary Variables for Detection

In [48]:
# Score for the number of frames the eyes are closed
eye_score = 0

# Threshold for the number of frames the eyes are closed
eye_score_threshold = 6

# Flag for the Eye Status
eyes_closed = False

# Counter for the number of frames the face is not detected
face_not_detected_counter = 0

# Threshold for the number of frames the face is not detected
face_not_detected_threshold = 20

# Flag for the Face Status
face_not_detected = False

# Flag for the Alarm Status
alarm_playing = False


# Functions

## play_alarm()

#### Plays the alarm indefinetely if it is not already playing

In [49]:
def play_alarm():
    global alarm_playing
    if not alarm_playing:
        sound.play(-1)
        alarm_playing = True

## stop_alarm()
#### Stops the alarm if it is playing

In [50]:
def stop_alarm():
    global alarm_playing
    if alarm_playing:
        sound.stop()
        alarm_playing = False                                                                              

## check_face()
#### Checks if the driver's face is visible

In [51]:
def check_face(face):
    if len(face) == 0:
        global face_not_detected_counter, face_not_detected_threshold, face_not_detected
        face_not_detected_counter += 1
    else:
        face_not_detected_counter = 0
    
    if face_not_detected_counter > face_not_detected_threshold:
        face_not_detected = True
    else:
        face_not_detected = False

In [52]:
cap = cv2.VideoCapture(0)

# List to store the processing time of each frame
frame_processing_times = []


while True:
    
    # Start timer, the timer is used to calculate the processing time of each frame
    start_time = time.time()
    
    
    # Read the frame
    ret, frame = cap.read()
    
    # Frame hight, Frame width
    fh, fw = frame.shape[:2]
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect the Face
    face = face_classifier.detectMultiScale(gray, minNeighbors=20, scaleFactor=1.1, minSize=(25, 25))
    
    # Check if the Face is detected
    if len(face) == 0:
        
        check_face(face)
        
        if face_not_detected:
            cv2.putText(frame, "Face Not Detected", (10, fh-20), font, 1, (0, 0, 255), 1, cv2.LINE_AA)
            
            if not alarm_playing:
                play_alarm()
        else:
            if alarm_playing:
                stop_alarm()

    
    
    else:
         
        # Detect the Left Eye (gray, minNeighbors = 50, scaleFactor = 1.05, minSize = (50, 50))
        left_eye = left_eye_classifier.detectMultiScale(gray)
        
        # Detect the Right Eye
        right_eye = right_eye_classifier.detectMultiScale(gray)
        
        
        for (x, y, w, h) in face:
            # Draw a Rectangle around the Face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 1)
        
        
        for (x, y, w, h) in left_eye:
            # Crop the Left Eye from the Frame
            eye = frame[y:y+h, x:x+w]
            
            # Draw a Rectangle around the Left Eye
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 1)
            
            # Convert the Left Eye to Grayscale
            eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
            
            # Resize the Left Eye to 64x64 for the Model
            eye = cv2.resize(eye, (64, 64))
            
            # Normalize the Left Eye between 0 and 1
            eye = eye/255
            
            # Reshape the Left Eye to 64x64x-1
            eye = eye.reshape(64, 64, -1)
            
            # Expand the Left Eye dimension
            eye = np.expand_dims(eye, 0)
            
            # Predict the Left Eye
            lpred = model.predict(eye)
            
        
        for (x, y, w, h) in right_eye:
            # Crop the Right Eye from the Frame
            eye = frame[y:y+h, x:x+w]
            
            # Draw a Rectangle around the Right Eye
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 1)
            
            # Convert the Right Eye to Grayscale
            eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
            
            # Resize the Right Eye to 64x64 for the Model
            eye = cv2.resize(eye, (64, 64))
            
            # Normalize the Right Eye between 0 and 1
            eye = eye/255
            
            # Reshape the Right Eye to 64x64x-1
            eye = eye.reshape(64, 64, -1)
            
            # Expand the Right Eye dimension
            eye = np.expand_dims(eye, 0)
             
            # Predict the Right Eye
            rpred = model.predict(eye)
            
        
        # Check if the Eyes are Closed according to the Model (0: Closed, 1: Open)
        
        # If either Eye is Closed
        if (lpred <= 0.5 or rpred <= 0.5):
            # Increment the Eye Score by 1
            eye_score+=1
            cv2.putText(frame, "Eyes Closed", (10, fh-150), font, 1, (0, 0, 255), 1, cv2.LINE_AA)
        
        # If both Eyes are Open
        elif (lpred > 0.5 and rpred > 0.5):
            # Decrement the Eye Score by 1
            eye_score-=1
            cv2.putText(frame, "Eyes Open", (10, fh-150), font, 1, (0, 255, 0), 1, cv2.LINE_AA)
        
        # If the Eye Score is less than 0, set it to 0
        if(eye_score < 0):
            eye_score = 0
        
        # Display the Eye Score
        cv2.putText(frame, f"Eyes Score: {eye_score}", (10, fh-100), font, 1, (255, 255, 0), 1, cv2.LINE_AA)
        
        # If the Eye Score is > the Eye Score Threshold, play the Alarm
        if(eye_score > eye_score_threshold):
            if not alarm_playing:
                play_alarm()
                
        # If the Eye Score is <= the Eye Score Threshold, stop the Alarm
        else:
            if alarm_playing:
                stop_alarm()
        
        
    # End timer
    end_time = time.time()
    
    # Calculate the time
    time_diff = end_time - start_time
    frame_processing_times.append(time_diff)
    
    
    # Display the Frame
    cv2.imshow('frame', frame)
    
    # Break the Loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the Camera and Destroy the Windows
cap.release()
cv2.destroyAllWindows()

# Stop the alarm if it is playing
if alarm_playing:
    stop_alarm()

1/1 [==============================] - 0s 23ms/step


In [53]:
# Average Frame Processing Time
print(f"Average Frame Processing Time: {np.mean(frame_processing_times)} seconds")

Average Frame Processing Time: 0.2976340899502274 seconds


## Memory Utilization:
#### ~ 700 MB


## CPU Utilization: Intel(R) Core(TM) i5-8600K CPU @ 3.60GHz 
#### ~ 60 %

## GPU Utilization: Nvidia RTX 2060 6GB VRAM
#### ~ 6 %